In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import *
import xgboost as xgb
import optuna
from sklearn.ensemble import VotingRegressor
from lightgbm import LGBMRegressor, early_stopping
from xgboost import XGBRegressor
import pickle

from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import root_mean_squared_log_error, root_mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold

import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [2]:
df = pd.read_csv("trainable_df.csv")

In [3]:
df

,Health Conscious Level,SCALER_Annual_Income,SCALER_Credit_Score,SCALER_IsNull_Annual_Income,SCALER_Money_Handling_Level,SCALER_Money_Handling_Level1,SCALER_Money_Per_Head,SCALER_Growth,SCALER_Credit_by_Score,SCALER_Determinstic,SCALER_Growth1,SCALER_Feedback1,SCALER_Previous_Claims_MEDIAN_Premium_Amount,SCALER_IsNull_Health_Score,SCALER_Previous_Claims_MEAN_Premium_Amount,SCALER_Previous_Claims,SCALER_Previous_Claims_STD_Premium_Amount,SCALER_IsNull_Customer_Feedback,SCALER_Previous_Claims_MAX_Premium_Amount,SCALER_Feedback3,SCALER_IsNull_Previous_Claims,SCALER_IsNull_Marital_Status,SCALER_Health_Score,SCALER_Health_Risk_Score,SCALER_Feedback2,SCALER_CreditInsurance,SCALER_Sin_Year,SCALER_IsNull_Credit_Score,SCALER_Health_Age_Interaction,SCALER_Total_Nulls,SCALER_ENCODED_Policy_Start_Date_-_Year,SCALER_ENCODED_Policy_Start_Date_-_Quarter,SCALER_Feedback4,SCALER_IsNull_Number_of_Dependents,SCALER_IsNull_Occupation,SCALER_Health_Conscious_Level1,SCALER_Sin_Month,SCALER_Policy_Start_Date_-_Month,SCALER_Health_Conscious_Level,SCALER_Health_Conscious_Level_MEAN_Premium_Amount,SCALER_Health_Conscious_Level_MEDIAN_Premium_Amount,SCALER_Number_of_Dependents_MEAN_Premium_Amount,SCALER_Number_of_Dependents_MEDIAN_Premium_Amount,SCALER_Number_of_Dependents_STD_Premium_Amount,SCALER_Insurance_Duration_MEAN_Premium_Amount,SCALER_Insurance_Duration_MEDIAN_Premium_Amount,SCALER_Health_Conscious_Level_MAX_Premium_Amount,SCALER_Credit_Health_Score,SCALER_Occupation_MEDIAN_Premium_Amount,SCALER_Occupation_MEAN_Premium_Amount,SCALER_Occupation_MAX_Premium_Amount,SCALER_Previous_Claims_MIN_Premium_Amount,SCALER_Insurance_Duration_MAX_Premium_Amount,SCALER_ENCODED_Occupation_Self-Employed,SCALER_Age,SCALER_Insurance_Duration_STD_Premium_Amount,Premium Amount
0,4,-0.391308,-0.978541,0.0,-0.498665,-0.174446,-0.069759,-0.323104,-1.087379,-0.065518,-0.257205,-0.404860,52.0,0.0,57.590036,1.0,30.472002,0.0,-4.5,0.00,0.0,0.0,-0.122626,0.122626,-0.725022,-0.347664,0.000000,0.0,-0.600824,-0.5,0.333333,0.7,-0.422991,0.0,0.0,-0.014024,-1.469576e-15,1.2,0.023383,0.0,0.095653,0.000,0.540228,-0.739699,0.00,-0.500000,-0.502054,-0.333333,-0.619544,0.4,0.0,0.222222,1.0,-0.956522,-2.154674,0.000000,2869.0
1,2,0.199725,0.403433,0.0,0.377397,0.111496,-0.047042,0.468789,0.556634,0.216699,0.044642,0.231642,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,-0.520317,0.520317,0.173298,-0.524112,0.000000,0.0,-0.389642,0.0,0.333333,0.5,-0.266722,0.0,1.0,-0.384137,0.000000e+00,0.0,-1.208313,-1.8,0.000000,0.125,0.000000,0.458999,1.50,0.833333,-0.289535,0.000000,0.000000,0.0,0.0,0.333333,0.0,-0.086957,-0.781814,0.469401,1483.0
2,4,0.033693,0.137339,0.0,0.098726,0.032674,-0.137197,-0.264940,0.355987,0.516358,0.865007,0.697934,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.50,0.0,0.0,1.267999,-1.267999,1.181256,-0.334206,0.000000,1.0,0.177804,0.0,0.333333,0.6,2.616359,0.0,0.0,0.136870,-7.347881e-16,0.6,0.023383,0.0,0.000000,0.125,0.000000,-0.557842,0.00,-0.500000,1.394813,-0.333333,-0.619544,0.4,0.0,0.333333,1.0,-0.782609,0.000000,0.000000,567.0
3,3,3.210417,-1.000000,0.0,1.821453,5.342896,2.640168,2.462638,-0.501618,6.135441,3.336871,1.168943,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,-0.25,0.0,0.0,-0.782364,0.782364,-0.729443,-0.905794,1.000000,0.0,-0.837931,0.0,0.666667,0.9,-0.636346,0.0,1.0,-0.280271,0.000000e+00,0.0,-0.976617,-0.8,0.631238,0.250,3.658057,-1.483870,-2.75,0.500000,-0.891222,-0.333333,-0.619544,0.4,0.0,-0.666667,1.0,-0.869565,-0.132030,0.000000,765.0
4,3,0.417595,-0.008584,0.0,0.460305,0.428567,1.247936,0.302539,0.245955,1.288240,0.549980,-0.051403,0.0,0.0,-1.000000,-1.0,-1.000000,0.0,1.0,-0.50,0.0,0.0,-0.248381,0.248381,-0.525199,-0.148785,0.753709,0.0,-0.602579,-0.5,-0.333333,-0.1,-0.463660,0.0,0.0,-0.301274,-1.469576e-15,1.2,-0.976617,-0.8,0.095653,0.000,0.540228,0.032506,0.00,0.500000,-0.167311,-0.333333,-0.619544,0.4,0.0,-0.333333,1.0,-0.869565,0.218186,0.000000,2022.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

#
---
#

# Just Checking!

In [4]:
def return_splits(ddf, feature_name, target_name):
    return [ddf[ddf[feature_name] == i][target_name] for i in ddf[feature_name].unique()]

def give_stats_analysis(df, target_column_name):
    ddf = df.copy()
    ddf = ddf.dropna()

    features = []
    tests = []
    stats = []
    pvals = []
    verdict = []
    count = 0

    target = ddf[target_column_name]
    for i in ddf.columns:
        features.append(i)
        feature = ddf[i]
        
        if (feature.dtype == "O" and (target.dtype == "float" or target.dtype == "int")) or (target.dtype == "O" and (feature.dtype == "float" or feature.dtype == "int")):
            stat, pval, *_ = kruskal(*return_splits(ddf, feature.name, target.name))
            tests.append("Kruskal-Wallis")
            stats.append(stat)
            pvals.append(pval)
            
        
        elif (feature.dtype == "float" or feature.dtype == "int") and (target.dtype == "float" or target.dtype == "int"):
            stat, pval, *_ = spearmanr(feature, target)
            tests.append("SpearmanR")
            stats.append(stat)
            pvals.append(pval)

        elif feature.dtype == "O" and target.dtype == "O":
            stat, pval, *_ = chi2_contingency(pd.crosstab(feature, target))
            tests.append("Chi-Square")
            stats.append(stat)
            pvals.append(pval)
        
        else:
            tests.append(np.nan)
            stats.append(np.nan)
            pvals.append(np.nan)
        
        if pval <= 0.05:
            verdict.append("There is Relationship")
        else:
            verdict.append("There is NO Relationship")

        print(f"{feature.name} ■■■ {target_column_name}".ljust(50, "-")+"✅")
    
    return pd.DataFrame({
        "Feature" : features,
        "Target" : [target_column_name]*ddf.shape[1],
        "Statistic Test" : tests,
        "Test Statistic" : stats,
        "P-Value" : pvals,
        "Verdict" : verdict
    }).sort_values(by="P-Value")

In [5]:
give_stats_analysis(df, "Premium Amount")

Health Conscious Level ■■■ Premium Amount---------✅
SCALER_Annual_Income ■■■ Premium Amount-----------✅
SCALER_Credit_Score ■■■ Premium Amount------------✅
SCALER_IsNull_Annual_Income ■■■ Premium Amount----✅
SCALER_Money_Handling_Level ■■■ Premium Amount----✅
SCALER_Money_Handling_Level1 ■■■ Premium Amount---✅
SCALER_Money_Per_Head ■■■ Premium Amount----------✅
SCALER_Growth ■■■ Premium Amount------------------✅
SCALER_Credit_by_Score ■■■ Premium Amount---------✅
SCALER_Determinstic ■■■ Premium Amount------------✅
SCALER_Growth1 ■■■ Premium Amount-----------------✅
SCALER_Feedback1 ■■■ Premium Amount---------------✅
SCALER_Previous_Claims_MEDIAN_Premium_Amount ■■■ Premium Amount✅
SCALER_IsNull_Health_Score ■■■ Premium Amount-----✅
SCALER_Previous_Claims_MEAN_Premium_Amount ■■■ Premium Amount✅
SCALER_Previous_Claims ■■■ Premium Amount---------✅
SCALER_Previous_Claims_STD_Premium_Amount ■■■ Premium Amount✅
SCALER_IsNull_Customer_Feedback ■■■ Premium Amount✅
SCALER_Previous_Claims_MAX_Pre

,Feature,Target,Statistic Test,Test Statistic,P-Value,Verdict
1,SCALER_Annual_Income,Premium Amount,SpearmanR,-0.061831,0.000000e+00,There is Relationship
2,SCALER_Credit_Score,Premium Amount,SpearmanR,-0.036687,0.000000e+00,There is Relationship
3,SCALER_IsNull_Annual_Income,Premium Amount,SpearmanR,-0.065399,0.000000e+00,There is Relationship
4,SCALER_Money_Handling_Level,Premium Amount,SpearmanR,-0.072097,0.000000e+00,There is Relationship
6,SCALER_Money_Per_Head,Premium Amount,SpearmanR,-0.053422,0.000000e+00,There is Relationship
5,SCALER_Money_Handling_Level1,Premium Amount,SpearmanR,-0.048668,0.000000e+00,There is Relationship
7,SCALER_Growth,Premium Amount,SpearmanR,-0.055000,0.000000e+00,There is Relationship
8,SCALER_Credit_by_Score,Premium Amount,SpearmanR,-0.054850,0.000000e+00,There is Relationship
12,SCALER_Previous_Claims_MEDIAN_Premium_Amount,Premium Amount,SpearmanR,0.037268,0.000000e+00,There is Relationship
9,SCALER_Determinstic,Premium Amount,SpearmanR,-0.056869,0.000000e+00,There is Relationship


In [6]:
df.isnull().sum()

Health Conscious Level                                      0
SCALER_Annual_Income                                        0
SCALER_Credit_Score                                         0
SCALER_IsNull_Annual_Income                                 0
SCALER_Money_Handling_Level                                 0
SCALER_Money_Handling_Level1                                0
SCALER_Money_Per_Head                                       0
SCALER_Growth                                               0
SCALER_Credit_by_Score                                      0
SCALER_Determinstic                                         0
SCALER_Growth1                                              0
SCALER_Feedback1                                            0
SCALER_Previous_Claims_MEDIAN_Premium_Amount                0
SCALER_IsNull_Health_Score                                  0
SCALER_Previous_Claims_MEAN_Premium_Amount                  0
SCALER_Previous_Claims                                      0
SCALER_P

#
---
#

# Removing non related columns

In [7]:
# df.drop(columns=["SCALER_ENCODED_Marital_Status_Married", "SCALER_ENCODED_Customer_Feedback", "SCALER_PC2_Meaningless_df", "SCALER_PC1_Meaningless_df", "SCALER_ENCODED_Occupation_Unemployed"], inplace=True)

#
---
#

# Spliting Data

In [8]:
train = df.iloc[:1200000, :]
test = df.iloc[1200000:, :]

train.shape, test.shape

((1200000, 57), (800000, 57))

In [9]:
X = train.drop(columns="Premium Amount")
Y = train["Premium Amount"]
Y_log = np.log1p(Y)

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
x_train, x_validate, y_train, y_validate = train_test_split(X, Y, test_size=10000)

In [12]:
x_validate.shape

(10000, 56)

In [13]:
test.drop(columns="Premium Amount", inplace=True)

In [14]:
test.shape

(800000, 56)

In [15]:
x_validate.head(3)

,Health Conscious Level,SCALER_Annual_Income,SCALER_Credit_Score,SCALER_IsNull_Annual_Income,SCALER_Money_Handling_Level,SCALER_Money_Handling_Level1,SCALER_Money_Per_Head,SCALER_Growth,SCALER_Credit_by_Score,SCALER_Determinstic,SCALER_Growth1,SCALER_Feedback1,SCALER_Previous_Claims_MEDIAN_Premium_Amount,SCALER_IsNull_Health_Score,SCALER_Previous_Claims_MEAN_Premium_Amount,SCALER_Previous_Claims,SCALER_Previous_Claims_STD_Premium_Amount,SCALER_IsNull_Customer_Feedback,SCALER_Previous_Claims_MAX_Premium_Amount,SCALER_Feedback3,SCALER_IsNull_Previous_Claims,SCALER_IsNull_Marital_Status,SCALER_Health_Score,SCALER_Health_Risk_Score,SCALER_Feedback2,SCALER_CreditInsurance,SCALER_Sin_Year,SCALER_IsNull_Credit_Score,SCALER_Health_Age_Interaction,SCALER_Total_Nulls,SCALER_ENCODED_Policy_Start_Date_-_Year,SCALER_ENCODED_Policy_Start_Date_-_Quarter,SCALER_Feedback4,SCALER_IsNull_Number_of_Dependents,SCALER_IsNull_Occupation,SCALER_Health_Conscious_Level1,SCALER_Sin_Month,SCALER_Policy_Start_Date_-_Month,SCALER_Health_Conscious_Level,SCALER_Health_Conscious_Level_MEAN_Premium_Amount,SCALER_Health_Conscious_Level_MEDIAN_Premium_Amount,SCALER_Number_of_Dependents_MEAN_Premium_Amount,SCALER_Number_of_Dependents_MEDIAN_Premium_Amount,SCALER_Number_of_Dependents_STD_Premium_Amount,SCALER_Insurance_Duration_MEAN_Premium_Amount,SCALER_Insurance_Duration_MEDIAN_Premium_Amount,SCALER_Health_Conscious_Level_MAX_Premium_Amount,SCALER_Credit_Health_Score,SCALER_Occupation_MEDIAN_Premium_Amount,SCALER_Occupation_MEAN_Premium_Amount,SCALER_Occupation_MAX_Premium_Amount,SCALER_Previous_Claims_MIN_Premium_Amount,SCALER_Insurance_Duration_MAX_Premium_Amount,SCALER_ENCODED_Occupation_Self-Employed,SCALER_Age,SCALER_Insurance_Duration_STD_Premium_Amount
553011,3,0.981138,-0.098712,0.0,0.991657,1.014121,2.165941,0.738410,0.177994,0.479698,1.112327,0.194842,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.25,0.0,0.0,-0.486810,0.486810,-0.543767,-0.180187,1.507418,1.0,-0.035957,0.0,-1.0,-0.9,-0.540766,0.0,0.0,-0.284849,-9.797174e-16,0.8,-0.976617,-0.8,0.095653,0.000,0.540228,0.032506,0.0,0.5,-0.420658,0.666667,0.380456,-0.6,0.0,-0.333333,0.0,0.652174,0.218186,-0.530599
311977,0,0.562696,0.085837,0.0,0.660257,0.523896,0.150050,0.889896,0.317152,1.130008,0.286109,1.622543,0.0,0.0,-1.0,-1.0,-1.0,0.0,1.0,-0.50,1.0,0.0,-0.685619,0.685619,1.138815,-0.347664,-1.000000,0.0,-0.720195,0.0,0.0,0.1,0.089208,0.0,0.0,-0.531742,0.000000e+00,0.0,2.568729,2.2,0.000000,0.125,0.000000,-0.557842,0.0,-2.0,-0.552106,-0.333333,-0.619544,0.4,0.0,0.333333,1.0,-0.652174,0.000000,0.000000
667212,5,-0.451807,0.403433,0.0,-0.417078,-0.415755,-0.342694,-0.287100,0.556634,-0.462221,-0.388791,-0.337743,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,1.0,0.0,0.430300,-0.430300,0.173298,-0.264673,-1.000000,0.0,1.195707,0.5,0.0,0.0,0.348125,0.0,1.0,0.708369,1.224647e-15,-1.0,0.000000,0.2,0.631238,0.250,3.658057,-0.557842,0.0,0.0,0.743574,-0.333333,-0.619544,0.4,0.0,0.333333,1.0,0.826087,0.000000,0.000000


#
---
#

# Base Model Building

In [16]:
# from xgboost import XGBRegressor
# from sklearn.metrics import root_mean_squared_log_error

In [17]:
# model = XGBRegressor(verbosity=3)

In [18]:
# model.fit(x_train, np.log(y_train))

In [19]:
# prediction = model.predict(x_validate)

In [20]:
# prediction[prediction < 0] = 0

In [21]:
# root_mean_squared_log_error(y_validate, prediction)

In [22]:
# prediction = pd.DataFrame({"id" : test.index, "Premium Amount" : model.predict(test)})
# prediction

In [23]:
# prediction.to_csv("Dileep's_Submission_Take3.csv", index=False)

#
---
#

# Hyper-tuning -- IT is taking too much time

In [24]:
# from sklearn.model_selection import RandomizedSearchCV

In [25]:
# esimator = XGBRegressor(n_jobs=-1, verbosity=3)

In [26]:
# param_grid = {
#     "n_estimators" : range(50, 1000),
#     "max_depth" : range(5, 30)
# }

In [27]:
# cv_model = RandomizedSearchCV(estimator=esimator, param_distributions=param_grid, verbose=3, n_jobs=-1, cv=2, n_iter=60)

In [28]:
# cv_model.fit(x_train, y_train)

#
---
#

# Optuna Hyper-parameter Tuning

In [29]:
def objective_lgbm(trial):
    params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'rmse',
        'n_estimators': trial.suggest_int('n_estimators', 300, 1500),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0),
        'device': 'gpu',
        'gpu_platform_id': 1,
        'gpu_device_id': 0,
        'verbose': 3,
    }

    model = LGBMRegressor(**params)
    model.fit(x_train, y_train, 
              eval_set=[(x_validate, y_validate)], 
              eval_metric='rmse', 
              callbacks=[early_stopping(stopping_rounds=200)])
    preds = model.predict(x_validate)
    return root_mean_squared_error(y_validate, preds)

# study_lgbm = optuna.create_study(direction='minimize')
# study_lgbm.optimize(objective_lgbm, n_trials=30)
# print("Best parameters for LightGBM:", study_lgbm.best_params)

In [30]:
# study_lgbm.best_params

In [31]:
def objective_xgb(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0),
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor'
    }

    model = XGBRegressor(**params)
    model.fit(x_train, y_train, 
              eval_set=[(x_validate, y_validate)],
              verbose=3)
    preds = model.predict(x_validate)
    return root_mean_squared_error(y_validate, preds)

# study_xgb = optuna.create_study(direction='minimize')
# study_xgb.optimize(objective_xgb, n_trials=30)
# print("Best parameters for XGBoost:", study_xgb.best_params)

In [32]:
# study_xgb.best_params

#
---
#

# Let's do Cross-Validation

In [33]:
X["SCALER_Previous_Claims_STD_Premium_Amount"].fillna(X["SCALER_Previous_Claims_STD_Premium_Amount"].median(), inplace=True)

In [34]:
test["SCALER_Previous_Claims_STD_Premium_Amount"].fillna(test["SCALER_Previous_Claims_STD_Premium_Amount"].median(), inplace=True)

In [35]:
# ===============================1.04 RMSLE================================
# ===============Best Parmeters for KNN Imputer Data=======================
lgb_params = {
    'n_estimators': 470,
     'learning_rate': 0.055186215312218706,
     'num_leaves': 131,
     'max_depth': 11,
     'min_child_samples': 100,
     'subsample': 0.7456958918734639,
     'colsample_bytree': 0.8071815211022829,
     'reg_alpha': 5.4821727206510005,
     'reg_lambda': 6.187384822358707
 }

# cat_params = {
#     'iterations': 3000, 
#     'learning_rate': 0.038365175314273574, 
#     'depth': 11, 
#     'l2_leaf_reg': 3.596285147607088, 
#     'bagging_temperature': 0.2618728648567565
# }

xgb_params = {
    'n_estimators': 711,
     'learning_rate': 0.018352261556416866,
     'max_depth': 9,
     'min_child_weight': 6,
     'subsample': 0.9301085945058237,
     'colsample_bytree': 0.6532668613763417,
     'reg_alpha': 9.92413527770934,
     'reg_lambda': 9.101727260657976
}
# ===============Best Parmeters for KNN Imputer Data=======================
# =========================================================================

# ===============================WASTE - 1.06 RMSLE=========================
# ===============Best Parmeters for MICE Imputer Data=======================
# lgb_params = {
#     'n_estimators': 1307,
#      'learning_rate': 0.04375816738515439,
#      'num_leaves': 115,
#      'max_depth': 12,
#      'min_child_samples': 68,
#      'subsample': 0.7671752683615976,
#      'colsample_bytree': 0.571860650734991,
#      'reg_alpha': 8.204021459357918,
#      'reg_lambda': 7.859184960261201
#  }

# cat_params = {
#     'iterations': 3000, 
#     'learning_rate': 0.038365175314273574, 
#     'depth': 11, 
#     'l2_leaf_reg': 3.596285147607088, 
#     'bagging_temperature': 0.2618728648567565
# }

# xgb_params = {
#     'n_estimators': 1542,
#      'learning_rate': 0.012690572771966613,
#      'max_depth': 8,
#      'min_child_weight': 9,
#      'subsample': 0.7556171177245923,
#      'colsample_bytree': 0.6264304912802068,
#      'reg_alpha': 4.469408656856697,
#      'reg_lambda': 7.444861387111808
# }
# ===============Best Parmeters for MICE Imputer Data=======================
# =========================================================================



In [36]:
from lightgbm import LGBMRegressor
# from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.metrics import root_mean_squared_log_error, root_mean_squared_error, r2_score


splitter = KFold(n_splits=10)
models = []


count = 1

folds_df = pd.DataFrame()

for train_index, test_index in splitter.split(X, Y_log):
    fold_xtrain  = X.iloc[train_index, :]
    fold_ytrain  = Y_log.iloc[train_index]
    fold_xtest  = X.iloc[test_index, :]
    fold_ytest  = Y_log.iloc[test_index]

    # LightGBM model
    lgbm_model = LGBMRegressor(**lgb_params, device='gpu', gpu_platform_id=1, gpu_device_id=0)

    # CatBoost model
    # cat_model = CatBoostRegressor(**cat_params, verbose=3, task_type= 'GPU')

    # XGBoost model
    xgb_model = XGBRegressor(**xgb_params, tree_method="gpu_hist", predictor= 'gpu_predictor')

    # Voting Regressor
    model = VotingRegressor(
        estimators=[
            ('lgbm', lgbm_model),
            # ('cat', cat_model),
            ('xgb', xgb_model)
        ]
    )

    model.fit(fold_xtrain, fold_ytrain)
    fold_pred = np.maximum(0, model.predict(fold_xtest))
    fold_rmsle = root_mean_squared_log_error(np.expm1(fold_ytest), np.expm1(fold_pred))
    print(f"Fold {count} RMSLE: {fold_rmsle}")
    models.append(model)

    folds_df[f"Fold {count}"] = model.predict(test)
    count += 1

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 4860
[LightGBM] [Info] Number of data points in the train set: 1080000, number of used features: 55
[LightGBM] [Info] Using requested OpenCL platform 1 device 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3050 6GB Laptop GPU, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 45 dense feature groups (49.44 MB) transferred to GPU in 0.073076 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 6.593622
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further spli

In [38]:
from lightgbm import LGBMRegressor
# from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression, Lasso
from xgboost import XGBRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.metrics import root_mean_squared_log_error, root_mean_squared_error, r2_score


splitter = KFold(n_splits=10)
models = []


count = 1

folds_df = pd.DataFrame()

for train_index, test_index in splitter.split(X, Y_log):
    fold_xtrain  = X.iloc[train_index, :]
    fold_ytrain  = Y_log.iloc[train_index]
    fold_xtest  = X.iloc[test_index, :]
    fold_ytest  = Y_log.iloc[test_index]

    # LightGBM model
    lgbm_model = LGBMRegressor(**lgb_params, device='gpu', gpu_platform_id=1, gpu_device_id=0)

    # CatBoost model
    # cat_model = CatBoostRegressor(**cat_params, verbose=3, task_type= 'GPU')

    # XGBoost model
    xgb_model = XGBRegressor(**xgb_params, tree_method="gpu_hist", predictor= 'gpu_predictor')
    xgb_model1 = XGBRegressor(**xgb_params, tree_method="gpu_hist", predictor= 'gpu_predictor')

    # Linear Regression
    lir_model = Lasso()

    # Voting Regressor
    # vote_model = VotingRegressor(
    #     estimators=[
    #         ('lgbm', lgbm_model),
    #         # ('cat', cat_model),
    #         ('xgb', xgb_model)
    #     ]
    # )
    
    model = StackingRegressor(
        estimators=[
            ('lgbm', lgbm_model),
            # ('cat', cat_model),
            ('xgb', xgb_model)
        ], final_estimator=xgb_model1
    )

    model.fit(fold_xtrain, fold_ytrain)
    fold_pred = np.maximum(0, model.predict(fold_xtest))
    fold_rmsle = root_mean_squared_log_error(np.expm1(fold_ytest), np.expm1(fold_pred))
    print(f"Fold {count} RMSLE: {fold_rmsle}")
    models.append(model)

    folds_df[f"Fold {count}"] = model.predict(test)
    count += 1

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 4860
[LightGBM] [Info] Number of data points in the train set: 1080000, number of used features: 55
[LightGBM] [Info] Using requested OpenCL platform 1 device 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3050 6GB Laptop GPU, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 45 dense feature groups (49.44 MB) transferred to GPU in 0.059929 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 6.593622
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further spli

KeyboardInterrupt: 

In [ ]:
# splitter = KFold(n_splits=10)

In [ ]:
# from sklearn.ensemble import GradientBoostingRegressor
# from xgboost import XGBRegressor
# from sklearn.metrics import root_mean_squared_log_error, root_mean_squared_error, r2_score
# gb_model = GradientBoostingRegressor(verbose=3, loss="absolute_error", learning_rate=0.5, n_estimators=150)
# xg_model = XGBRegressor(verbose=3, loss="absolute_error", learning_rate=0.075, n_estimators=100)

# count = 1

# folds_df = pd.DataFrame()

# for train_index, test_index in splitter.split(X, Y_log):
#     fold_xtrain  = X.iloc[train_index, :]
#     fold_ytrain  = Y_log.iloc[train_index]
#     fold_xtest  = X.iloc[test_index, :]
#     fold_ytest  = Y_log.iloc[test_index]

#     # print(fold_xtrain.shape, fold_ytrain.shape, fold_xtest.shape, fold_ytest.shape)

#     fold_ypred = xg_model.fit(fold_xtrain, fold_ytrain).predict(fold_xtest)
#     print(f"The Root mean squared log error for FOLD - {count} is --> {root_mean_squared_log_error(fold_ytest, fold_ypred)}")
    
#     folds_df[f"Fold {count}"] = xg_model.predict(test)
#     count += 1

In [37]:
with open('models.pkl', 'wb') as file:
    pickle.dump(models, file)

print("Cross-validation models saved!")

Cross-validation models saved!


In [38]:
# from sklearn.metrics import root_mean_squared_log_error, root_mean_squared_error, r2_score

# print(f"The Root mean squared log error is --> {np.exp(root_mean_squared_log_error(fold_ytest, fold_ypred))}")
# print(f"The R-Square is --> {r2_score(fold_ytest, fold_ypred)}")


In [39]:
folds_df.to_csv("folds_prediction.csv", index=False)

#
---
#

In [40]:
preds = pd.read_csv("folds_prediction.csv")
preds

,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Fold 6,Fold 7,Fold 8,Fold 9,Fold 10
0,6.551424,6.498545,6.601262,6.524946,6.573359,6.519237,6.479273,6.679409,6.592991,6.469150
1,6.693403,6.691399,6.719050,6.669937,6.733575,6.682848,6.725292,6.675935,6.723237,6.686640
2,6.669284,6.660493,6.636197,6.658148,6.655478,6.664165,6.689860,6.683136,6.643519,6.643531
3,6.687991,6.692937,6.685077,6.689452,6.694585,6.698281,6.695249,6.708025,6.699348,6.718604
4,6.601223,6.620842,6.594584,6.616084,6.613587,6.607813,6.592059,6.597258,6.573550,6.591645
...,...,...,...,...,...,...,...,...,...,...
799995,6.902434,6.905477,6.884256,6.885296,6.910605,6.916957,6.910586,6.877114,6.888830,6.883066
799996,6.268355,6.147964,6.171807,6.140232,6.172171,6.267633,6.156420,6.188502,6.198136,6.273045
799997,6.682500,6.651546,6.689899,6.652617,6.677367,6.664535,6.613153,6.620883,6.638718,6.661296
799998,6.718887,6.710274,6.686544,6.713961,6.712388,6.706915,6.681868,6.686890,6.705686,6.706521


In [41]:
pred = preds.mean(axis=1).apply(np.expm1)

In [42]:
prediction = pd.DataFrame({"id" : test.index, "Premium Amount" : pred})
prediction

,id,Premium Amount
0,1200000,697.517068
1,1200001,811.512707
2,1200002,779.848416
3,1200003,808.935779
4,1200004,734.730923
...,...,...
799995,1999995,987.770323
799996,1999996,490.974306
799997,1999997,775.853365
799998,1999998,813.841462


In [43]:
prediction.to_csv("Dileep's_Submission_Take34.csv", index=False)

In [ ]:
# Take 20 did after Feature engineering
# Before -- 821 Rank
# After -- 757

#
---
#

# Lets implement... `STACKING`

In [ ]:
X.fillna(X["SCALER_Previous_Claims_STD_Premium_Amount"].median(), inplace=True)

In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, IsolationForest
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.metrics import root_mean_squared_log_error

In [ ]:
lir_model = LinearRegression()
knn_model = KNeighborsRegressor()
rf_model = RandomForestRegressor(verbose=3, n_estimators=40)
gb_model = GradientBoostingRegressor(verbose=3, loss="absolute_error", learning_rate=0.5, n_estimators=250)
xgb_model = XGBRegressor(verbose=3)
ridge_model = Ridge()
if_model = IsolationForest(verbose=3)

In [ ]:
stacked_model = StackingRegressor(estimators=[
                    ("rf_model", rf_model),
                    # ("knn_model", knn_model),
                    ("xgb_model", xgb_model),
                    ("gb_model", gb_model),
                    # ("lir_model", lir_model)
                ], final_estimator=lir_model, verbose=3)

In [ ]:
gb_model.fit(X, np.log(Y))

In [ ]:
prediction = gb_model.predict(x_validate)

In [ ]:
# prediction[prediction < 0] = prediction.mean()

In [ ]:
root_mean_squared_log_error(y_validate, prediction)

In [ ]:
test.fillna(test["SCALER_Previous_Claims_STD_Premium_Amount"].median(), inplace=True)

In [ ]:
prediction = pd.DataFrame({"id" : test.index, "Premium Amount" : np.exp(gb_model.predict(test))})
prediction

In [ ]:
prediction.to_csv("Dileep's_Submission_Take14.csv", index=False)